In [2]:
import MySQLdb as db
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine

from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [3]:
import json

with open('/home/ej/github/news_config.json') as f:
    config = json.load(f)

db = db.connect(
    config['ADDR'],
    config['ID'],
    config['PW'],
    config['DB'],
    charset='utf8'
)

In [3]:
sql = 'SELECT * FROM VIEW'
view = pd.read_sql(sql, db)
view['count'] = 1

In [4]:
pivot2 = view.pivot(index='user_id',columns='article_id',values='count').fillna(0)
pivot = pivot2.copy()

In [5]:
def recommend_articles(pivot, user_id, how_many_to_consider=5, how_many_to_rec=5):
    raw = pivot.values
    spr = sparse.csr_matrix(raw)
    # 코사인 유사도 행렬
    sim = cosine_similarity(spr)

    similar_user_index = sim[user_id -
                             1].argsort()[-how_many_to_consider:][::-1][1:]
    similar_users_item = raw[similar_user_index]
    # 비슷한 유저들이 어떤 기사를 봤는지 평균을 냄
    item_vector = np.sum(similar_users_item, axis=0) / how_many_to_consider
    pivot.loc['item'] = item_vector

    user_row = pivot.loc[user_id]
    # 유저가 본 기사는 추천에서 제외
    user_seen = user_row[user_row != 0].index

    rec = pivot.loc['item']
    # 비슷한 유저들이 본 기사들
    rec_index = rec[rec != 0].index

    # 안 본 기사의 리스트를 구한 뒤
    not_seen_articles = set(rec_index) - set(user_seen)
    rec_target = rec[not_seen_articles]

    # 유사도가 비슷한 유저들이 봤던 기사 중 제일 값이 높은 애들을 추천
    return rec_target.sort_values(ascending=False)[:how_many_to_rec]

In [20]:
recommend_articles(pivot2.copy(),150)

article_id
5029    0.6
3189    0.6
3348    0.4
447     0.4
655     0.4
Name: item, dtype: float64

In [7]:
a = pivot.values
spr = sparse.csr_matrix(a)
sim = cosine_similarity(spr)

In [8]:
similar_user_index = sim[0].argsort()[-5:][::-1][1:]

In [9]:
similar_user = a[similar_user_index]

In [10]:
item_vector = np.sum(similar_user, axis=0) / 5
len(item_vector)

5255

In [11]:
pivot.loc['item'] = item_vector

In [12]:
pivot.loc[[1,'item']]

article_id,1,2,3,4,5,6,8,10,11,12,...,5603,5604,5605,5606,5607,5608,5609,5610,5611,5612
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
item,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0


In [13]:
tmp = pivot.loc[1]
user_seen = tmp[tmp!=0].index

In [14]:
len(user_seen)

64

In [15]:
rec = pivot.loc['item']
rec_index = rec[rec!=0].index

In [16]:
len(rec_index)

301

In [17]:
not_seen_list = set(rec_index) - set(user_seen)

In [18]:
rec_target = rec[not_seen_list]

In [19]:
# ID 1인 유저를 위한 추천 리스트
rec_target.sort_values(ascending=False)[:5]

article_id
1585    0.4
3687    0.4
2813    0.4
760     0.4
4600    0.4
Name: item, dtype: float64